In [1]:
import pandas as pd
from itertools import combinations

evident = pd.read_csv("../data/data_w_evidence.csv").dropna()
evident = evident.astype({"evident_pubmed_id": int})
print(len(evident))
evident.head(2)

94


,data_source,query_id,description,query,label,evident_pubmed_id
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,23076891
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0,30480774


## Keywords

In [2]:
keywords_files = [
    "../data/pubmed_search/pubmed_search_abstracts_keywords.csv",
    "../data/google_search/google_search_abstracts_keywords_part_1.csv",
    "../data/google_search/google_search_abstracts_keywords_part_2.csv",
    "../data/biomed_explorer/biomed_explorer_abstracts_keywords.csv"
]

In [3]:
def prepare_result(files):
    result = {}

    for file in files:

        if "2" in file:
            continue

        if file.startswith("google"):
            df = pd.concat((
                pd.read_csv(file), pd.read_csv(file.replace("1", "2"))
            )).astype({"query_id": str})
        else:
            df = pd.read_csv(file).astype({"query_id": str})

        df = df.merge(evident)

        cnt = 0

        for _, group in df.groupby(["query_id", "data_source"]):
            evident_id = group.evident_pubmed_id.tolist()[0]
            has_evident = bool(len(group.query(f"pubmed_id == {evident_id}")))

            if has_evident:
                cnt += 1

        retriever = file.split("/")[2]
        result[retriever] = cnt
    return result

In [9]:
keywords_result = prepare_result(keywords_files)

## Question

In [6]:
question_files = [
    "../data/pubmed_search/pubmed_search_abstracts_question.csv",
    "../data/google_search/google_search_abstracts_question_part_1.csv",
    "../data/google_search/google_search_abstracts_question_part_2.csv",
    "../data/biomed_explorer/biomed_explorer_abstracts_question.csv"
]

In [8]:
question_result = prepare_result(question_files)

In [19]:
pd.DataFrame(
    data={
        "Keywords": keywords_result.values(), 
        "Question": question_result.values()
    },
    index=keywords_result.keys()
)

,Keywords,Question
pubmed_search,10,7
google_search,56,39
biomed_explorer,41,39
